<a href="https://colab.research.google.com/github/brendanpshea/colab-utilities/blob/main/SQL_Select_Quiz)Improved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import sqlite3
import pandas as pd
import requests
import tempfile
import json
from IPython.display import display, HTML, clear_output
from ipywidgets import Textarea, Button, VBox, Layout
import os

def create_default_database(db_path):
    """
    Creates a default database with a sci-fi fantasy theme for a space academy.

    Args:
        db_path (str): Path to the SQLite database file.
    """
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()

        # Create Students table
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS students (
            student_id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            species TEXT NOT NULL,
            home_planet TEXT NOT NULL
        )
        ''')

        # Create Classes table
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS classes (
            class_id INTEGER PRIMARY KEY,
            class_name TEXT NOT NULL,
            instructor TEXT NOT NULL,
            max_capacity INTEGER NOT NULL
        )
        ''')

        # Create Enrollments table
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS enrollments (
            enrollment_id INTEGER PRIMARY KEY,
            student_id INTEGER,
            class_id INTEGER,
            enrollment_date TEXT NOT NULL,
            FOREIGN KEY (student_id) REFERENCES students (student_id),
            FOREIGN KEY (class_id) REFERENCES classes (class_id)
        )
        ''')

        # Insert sample data
        cursor.executemany('INSERT INTO students (name, species, home_planet) VALUES (?, ?, ?)',
                           [('Zorp Xylax', 'Zorlack', 'Xenon-7'),
                            ('Lira Starwhisper', 'Elf', 'Eldoria'),
                            ('Blip Neutron', 'Robot', 'Mechanica'),
                            ('Galax Stormrider', 'Human', 'Earth'),
                            ('Nebula Moonshadow', 'Lunarian', 'Luna')])

        cursor.executemany('INSERT INTO classes (class_name, instructor, max_capacity) VALUES (?, ?, ?)',
                           [('Astro-Navigation 101', 'Captain Pulsar', 30),
                            ('Xenobiology', 'Dr. Tentacula', 25),
                            ('Quantum Mechanics', 'Professor Schrodinger', 20),
                            ('Telepathy for Beginners', 'Madame Psyche', 15),
                            ('Lightsaber Combat', 'Master Lumina', 10)])

        cursor.executemany('INSERT INTO enrollments (student_id, class_id, enrollment_date) VALUES (?, ?, ?)',
                           [(1, 1, '2345-09-01'), (1, 3, '2345-09-02'),
                            (2, 2, '2345-09-01'), (2, 4, '2345-09-03'),
                            (3, 1, '2345-09-02'), (3, 3, '2345-09-04'),
                            (4, 5, '2345-09-01'), (4, 2, '2345-09-03'),
                            (5, 4, '2345-09-02'), (5, 5, '2345-09-04')])

        conn.commit()

def create_quiz_database(db_path):
    """
    Creates a new database to store quiz questions and answers.

    Args:
        db_path (str): Path to the new SQLite database file.
    """
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS quiz_questions (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            question TEXT NOT NULL,
            answer TEXT NOT NULL,
            problem_set TEXT NOT NULL,
            source_database TEXT NOT NULL
        )
        ''')
        conn.commit()

def add_quiz_question(db_path, question, answer, problem_set, source_database):
    """
    Adds a new quiz question to the database.

    Args:
        db_path (str): Path to the SQLite database file.
        question (str): The question text.
        answer (str): The correct SQL query answer.
        problem_set (str): The problem set this question belongs to.
        source_database (str): The source database for this question.
    """
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        cursor.execute('''
        INSERT INTO quiz_questions (question, answer, problem_set, source_database)
        VALUES (?, ?, ?, ?)
        ''', (question, answer, problem_set, source_database))
        conn.commit()

def get_quiz_questions(db_path, problem_set=None, source_database=None):
    """
    Retrieves quiz questions from the database.

    Args:
        db_path (str): Path to the SQLite database file.
        problem_set (str, optional): Filter questions by problem set.
        source_database (str, optional): Filter questions by source database.

    Returns:
        list: A list of tuples containing (question, answer) pairs.
    """
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        query = "SELECT question, answer FROM quiz_questions"
        conditions = []
        params = []
        if problem_set:
            conditions.append("problem_set = ?")
            params.append(problem_set)
        if source_database:
            conditions.append("source_database = ?")
            params.append(source_database)
        if conditions:
            query += " WHERE " + " AND ".join(conditions)
        cursor.execute(query, params)
        return cursor.fetchall()

def validate_questions(conn, answers):
    """
    Validates each answer query to ensure it is executable and a SELECT statement.

    Args:
        conn (sqlite3.Connection): The database connection object.
        answers (list): A list of SQL queries representing the answers to the questions.

    Returns:
        tuple: A tuple containing a boolean indicating if all queries are valid and a list of valid queries.
    """
    valid_queries = []
    for i, query in enumerate(answers, start=1):
        try:
            if not query.strip().lower().startswith('select'):
                raise ValueError(f"Query {i} is not a SELECT statement.")
            conn.execute(query)
            valid_queries.append(query)
        except Exception as e:
            display(HTML(f"<div style='color: red;'>Error in query {i}: {e}</div>"))
            return False, []
    return True, valid_queries

def get_table_schemas(conn):
    """
    Retrieves the schema information for all tables in the database.

    Args:
        conn (sqlite3.Connection): The database connection object.

    Returns:
        list: A list of tuples containing table names and their respective column information.
    """
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    schemas = []
    for table in tables:
        table_name = table[0]
        cursor.execute(f"PRAGMA table_info({table_name})")
        columns = cursor.fetchall()
        schemas.append((table_name, columns))
    return schemas

def render_table_schemas(schemas):
    """
    Renders the database schema information in a compact format with HTML formatting.

    Args:
        schemas (list): A list of tuples containing table names and their respective column information.

    Returns:
        str: The rendered HTML string representing the database schema.
    """
    schema_html = "<h2>Database Schema:</h2>"
    schema_html += "<ol>"
    for table_name, columns in schemas:
        column_info = ", ".join(f"{column[1]} {column[2]}" for column in columns)
        schema_html += f"<li><b>{table_name}</b> ({column_info})</li>"
    schema_html += "</ol>"

    schema_html += "<h3>Sample queries</h3>"
    sample_query = f'SELECT * FROM {schemas[0][0]} returns all rows and columns from {schemas[0][0]}.'
    schema_html += sample_query + "<br>"
    sample_query = f'SELECT {schemas[0][1][1][1]} FROM {schemas[0][0]} selects a specific column.'
    schema_html += sample_query
    return schema_html

def sql_select_quiz(quiz_db_path, source_db_path=None, problem_set=None, source_database=None):
    """
    Iterates through SQL SELECT questions from the quiz database, allowing the user to submit queries
    against a provided SQLite database or the default sci-fi fantasy database.

    Args:
        quiz_db_path (str): Path to the SQLite database file containing quiz questions.
        source_db_path (str, optional): Path to the SQLite database file to run queries against.
        problem_set (str, optional): Filter questions by problem set.
        source_database (str, optional): Filter questions by source database.
    """
    if source_db_path is None:
        source_db_path = 'space_academy.db'
        if not os.path.exists(source_db_path):
            create_default_database(source_db_path)

    questions_and_answers = get_quiz_questions(quiz_db_path, problem_set, source_database)
    if not questions_and_answers:
        display(HTML("<div>No questions found for the specified criteria.</div>"))
        return

    questions, answers = zip(*questions_and_answers)

    with sqlite3.connect(source_db_path) as conn:
        valid, valid_answers = validate_questions(conn, answers)
        if not valid:
            display(HTML("<div>Please correct the errors in your SQL queries before proceeding.</div>"))
            return

        question_index = 0

        def display_current_question():
            """
            Displays the current question and resets the UI for answer submission.
            """
            clear_output(wait=True)
            schemas = get_table_schemas(conn)
            display(HTML(render_table_schemas(schemas)))
            question_html = f"<h3>SQL Question {question_index + 1}:</h3><p>{questions[question_index]}</p>"
            display(HTML(question_html))

            text_area.value = ''
            submit_button.layout.visibility = 'visible'
            next_button.layout.visibility = 'hidden'
            retry_button.layout.visibility = 'hidden'
            display(query_widget)

        def submit_query(button):
            """
            Handles the submission of the user's query and compares it to the correct answer.
            """
            nonlocal question_index
            user_query = text_area.value.strip()

            if not user_query.lower().startswith('select'):
                display(HTML("<div style='color: red;'><strong>Error:</strong> Please enter a valid SELECT query.</div>"))
                return

            try:
                user_result = pd.read_sql_query(user_query, conn)
                correct_query = answers[question_index]
                correct_result = pd.read_sql_query(correct_query, conn)

                if user_result.equals(correct_result):
                    display(HTML("<div style='color: green;'><strong>Correct!</strong> Your query produced the expected result.</div>"))
                    submit_button.layout.visibility = 'hidden'
                    next_button.layout.visibility = 'visible'
                else:
                    display(HTML("<div style='color: red;'><strong>Incorrect.</strong> Your query did not produce the expected result. Please try again.</div>"))

                display(HTML("<h4>Your Results:</h4>"))
                display(user_result)
                display(HTML("<h4>Expected Results:</h4>"))
                display(correct_result)

            except Exception as e:
                display(HTML(f"<div style='color: red;'><strong>Error:</strong> {str(e)}</div>"))

        def next_question(button):
            """
            Advances to the next question if available.
            """
            nonlocal question_index
            question_index += 1
            if question_index < len(questions):
                display_current_question()
            else:
                submit_button.layout.visibility = 'hidden'
                next_button.layout.visibility = 'hidden'
                retry_button.layout.visibility = 'hidden'
                display(HTML("<div>All questions completed. Well done!</div>"))

        def retry_question(button):
            """
            Resets the interface for the user to retry the current question.
            """
            display_current_question()

        text_area = Textarea(value='', placeholder='Type your SQL query here...', description='Query:', layout=Layout(width='60%', height='100px'))
        submit_button = Button(description="Submit")
        next_button = Button(description="Next Question", layout=Layout(visibility='hidden'))
        retry_button = Button(description="Retry", layout=Layout(visibility='hidden'))

        submit_button.on_click(submit_query)
        next_button.on_click(next_question)
        retry_button.on_click(retry_question)

        query_widget = VBox([text_area, submit_button, retry_button, next_button])

        display_current_question()


def sql_select_quiz_url(quiz_db_url, source_db_url=None, problem_set=None, source_database=None):
    """
    Launches the SQL SELECT quiz using the provided database URLs.

    Args:
        quiz_db_url (str): URL of the SQLite database file containing quiz questions.
        source_db_url (str, optional): URL of the SQLite database file to run queries against.
        problem_set (str, optional): Filter questions by problem set.
        source_database (str, optional): Filter questions by source database.
    """
    with tempfile.NamedTemporaryFile(delete=False) as temp_quiz_db:
        quiz_db_path = temp_quiz_db.name
        response = requests.get(quiz_db_url)
        temp_quiz_db.write(response.content)

    if source_db_url:
        with tempfile.NamedTemporaryFile(delete=False) as temp_source_db:
            source_db_path = temp_source_db.name
            response = requests.get(source_db_url)
            temp_source_db.write(response.content)
    else:
        source_db_path = None  # This will trigger the use of the default database

    sql_select_quiz(quiz_db_path, source_db_path, problem_set, source_database)

# Example usage:
# create_quiz_database('quiz_questions.db')
# add_quiz_question('quiz_questions.db', 'List all student names and their species.', 'SELECT name, species FROM students', 'space_academy', 'space_academy.db')
# add_quiz_question('quiz_questions.db', 'How many students are enrolled in each class?', 'SELECT c.class_name, COUNT(e.student_id) as enrolled_students FROM classes c LEFT JOIN enrollments e ON c.class_id = e.class_id GROUP BY c.class_id', 'space_academy', 'space_academy.db')
# sql_select_quiz('quiz_questions.db', problem_set='space_academy')
# sql_select_quiz_url('http://example.com/quiz_db.sqlite', problem_set='space_academy')

In [12]:
create_quiz_database('quiz_questions.db')
add_quiz_question('quiz_questions.db', 'List all student names and their species.', 'SELECT name, species FROM students', 'space_academy', 'space_academy.db')
add_quiz_question('quiz_questions.db', 'How many students are enrolled in each class?', 'SELECT c.class_name, COUNT(e.student_id) as enrolled_students FROM classes c LEFT JOIN enrollments e ON c.class_id = e.class_id GROUP BY c.class_id', 'space_academy', 'space_academy.db')

In [13]:
sql_select_quiz('quiz_questions.db', problem_set='space_academy')

,name,species
0,Zorp Xylax,Zorlack
1,Lira Starwhisper,Elf
2,Blip Neutron,Robot
3,Galax Stormrider,Human
4,Nebula Moonshadow,Lunarian


,name,species
0,Zorp Xylax,Zorlack
1,Lira Starwhisper,Elf
2,Blip Neutron,Robot
3,Galax Stormrider,Human
4,Nebula Moonshadow,Lunarian
